In [7]:
import getexifdata
from PIL import Image
import glob, os
import json
import io

# process every dir in Places/
for base_path in glob.glob("../Places/*/"):
    
    print("processing place: " + base_path)

    ####################
    ###
    ### helper functions
    ###
    
    def strip_dir(fpath, dir):
        i = fpath.index(dir)+len(dir)
        return fpath[i:len(fpath)]

    def get_fname(fpath):
        return fpath.split("/")[-1]

    def get_flabel(fpath, ext):
        return get_fname(fpath).replace(ext, "")

    def find_audio_for_img(label):
        for fpath in glob.glob(base_path + "aud/*.mp3"):
            flabel = get_flabel(fpath, ".mp3")
            if flabel == label:
                return strip_dir(fpath, "/aud/")
        return None # no warning needed

    def get_gps_for_fpath(fpath):
        im = Image.open(fpath)
        edat = getexifdata.get_exif_data(im)
        lat, lng = getexifdata.get_lat_lon(edat)
        if lat == None or lng == None:
            #raise AttributeError("no gps data for image: " + fpath)
            return None
        else:
            return {"lat":lat, "lng":lng}

    try:
        to_unicode = unicode
    except NameError:
        to_unicode = str    
    
    def find_kml():
        return [strip_dir(fpath, "/kml/") for fpath in glob.glob(base_path + "kml/*.kml")]

    ####################
    ###
    ### sanity check
    ###

    # sanity check: every audio file should have a matching img
    def ensure_audio_img_match():
        # error if an audio file has no image. slow, but we should ensure this.
        for fpath in glob.glob(base_path + "aud/*.mp3"):
            match = False
            for imgpath in glob.glob(base_path + "img/*.jpg"):
                if get_flabel(imgpath, ".jpg") == get_flabel(fpath, ".mp3"):
                    match = True
            if not match:
                #raise FileNotFoundError("no image for audio file: " + fpath)
                return True

    # sanity check: every image should have GPS coordinates
    def ensure_img_gps():
        [get_gps_for_fpath(fpath) for fpath in glob.glob(base_path + "img/*.jpg")]

    ####################
    ###
    ### run
    ###

    ensure_img_gps()
    ensure_audio_img_match()

    # read json template
    with open(base_path + "info_template.json") as data_file:
        data = json.load(data_file)
        data["layers"] = find_kml()

        # add location for every image 
        for fpath in glob.glob(base_path + "img/*.jpg"):
            flabel = get_flabel(fpath, ".jpg")
            marker = {
                "label": flabel,
                "loc": get_gps_for_fpath(fpath),
                "img": get_fname(fpath),
                "aud": find_audio_for_img(flabel)
            }
            data["locations"].append(marker)

        # Write JSON file
        # http://stackoverflow.com/questions/12309269/how-do-i-write-json-data-to-a-file-in-python
        with io.open(base_path + "info.json", 'w', encoding='utf8') as outfile:
            str_ = json.dumps(data,
                              indent=4, sort_keys=True,
                              separators=(',', ':'), ensure_ascii=False)
            outfile.write(to_unicode(str_))


processing place: ../Places/CostaRica/
processing place: ../Places/RussianRiver/


In [8]:
strip_dir("/aud/nothing.mpo3", "/aud/")

'nothing.mpo3'

In [6]:
"../aud/nothing.mpo3".index("/aud/")

2